In [7]:
from src.ollama_client import OllamaClient
from src.database_client import DatabaseClient
import asyncio
import json
import logging
from typing import Dict, Any, Optional, Tuple
from datetime import datetime
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import re
import sqlglot
from sqlglot.expressions import Column
from sqlglot.errors import ParseError
import sqlglot
from sqlglot import expressions as exp

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class Pipeline:
    def __init__(self):
        self.ollama_base_url = "http://localhost:11434"
        self.database = DatabaseClient()
        #tem que estar em inglês
        self.table_schema = """
        CREATE TABLE IF NOT EXISTS metric (
            time TIMESTAMPTZ NOT NULL PRIMARY KEY, -- Metric timestamp
            fivegs_amffunction_rm_registeredsubnbr INT, -- Number of registered subscribers
            fivegs_amffunction_rm_regmobreq INT, -- Mobility registration update requests
            fivegs_amffunction_rm_regmobsucc INT, -- Successful mobility registration updates
            fivegs_amffunction_rm_regmobfail INT, -- Failed mobility registration updates
            fivegs_smffunction_sm_sessionnbr INT, -- Number of active sessions
            fivegs_smffunction_sm_pdusessioncreationreq INT, -- PDU session creation requests
            fivegs_smffunction_sm_pdusessioncreationsucc INT, -- Successful PDU session creations
            fivegs_ep_n3_gtp_indatapktn3upf INT, -- Uplink throughput on N3 interface
            fivegs_ep_n3_gtp_outdatapktn3upf INT, -- Downlink throughput on N3 interface
            fivegs_upffunction_upf_sessionnbr INT, -- Average number of PDU sessions
            fivegs_smffunction_upf_qos_flow_nbr INT, -- Number of active QoS flows
            fivegs_upffunction_sm_n4sessionreport INT, -- N4 session reports
            fivegs_upffunction_sm_n4sessionreportsucc INT, -- Successful N4 session reports
            fivegs_upffunction_sm_n4sessionestabreq INT, -- N4 session establishment requests
            fivegs_upffunction_sm_n4sessionestabfail INT, -- Failed N4 session establishments
            softmodern_bler_dl INT, -- Block error rate - downlink
            softmodern_bler_ul INT, -- Block error rate - uplink
            softmodern_rsrp INT, -- Received signal power
            pfcp_sessions_active INT -- Used for testing value changes
        );
        Get the current 5G network performance metrics, including throughput, latency and packet loss rates.
        """
        
    def call_ollama(self, model : str, prompt : str, system_prompt: str = None) -> str:

        try:
            url = f"{self.ollama_base_url}/api/generate"
            payload = {
                "model": model,
                "prompt":prompt,
                "stream":False,
                "options":{
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "num_predict": 1000
                }
            }
            if system_prompt:
                payload["system"] = system_prompt
            response = requests.post(url, json=payload, timeout=120)
            response.raise_for_status()

            return response.json()["response"].strip()
        except Exception as e:
            print(e)

    def translate_and_process_question(self, question: str) -> str:
        # self.table_schema
        system_prompt = """
        Você é um assistente especializado em tradução e processamento de perguntas para consultas SQL.
        Sua tarefa é:
        1. Traduzir a pergunta do português para o inglês
        2. Reformular a pergunta para ser mais específica e clara para geração de SQL
        3. Incluir contexto técnico relevante sobre métricas de redes 5G quando apropriado
        4. Manter o significado original mas torná-la mais precisa
        
        Responda apenas com a pergunta traduzida e processada, sem explicações adicionais.
        """
        
        prompt = f"""
        Pergunta original em português: "{question}"
        
        Traduza para inglês e processe para ser mais específica para geração de SQL sobre métricas de rede 5G.
        """
        
        logger.info("Etapa 1: Traduzindo e processando pergunta...")
        processed_question = self.call_ollama("llama3.2:latest", prompt, system_prompt)
        logger.info(f"Pergunta processada: {processed_question}")
        
        return processed_question
    
    def execute_sql_query(self, sql_query: str) -> Tuple[list, str]:
        try:
            logger.info("Etapa 3: Executando query SQL...")
            return self.database.execute(sql_query)                    
        except Exception as e:
            error_msg = f"Erro ao executar query SQL: {str(e)}"
            logger.error(error_msg)
            return [], error_msg

    def generate_sql_query(self, processed_question: str, max_retries: int = 3) -> str:
        system_prompt = """
        You are an expert SQL query generator for PostgreSQL. Generate precise SQL queries based on the user's question.
        Always use proper SQL syntax and consider performance optimizations.
        Focus on the specific metrics requested and use appropriate aggregations and filters.
        Return only the SQL query without any explanations or markdown formatting.
        """

        available_columns = self._extract_column_names()

        base_prompt = f"""
        Database schema:
        {self.table_schema}

        Available columns in 'metric' table:
        {', '.join(available_columns)}

        User question: {processed_question}

        Generate a SQL query to answer this question. Consider:
        - The table name is 'metric'
        - Use appropriate time filtering when needed
        - Apply proper aggregations (SUM, AVG, COUNT, etc.)
        - Include relevant columns based on the question
        - Use proper PostgreSQL syntax
        - ONLY use column names that exist in the available columns list above

        Important constraints:
        - Table name: 'metric'
        - Time column: 'time' (TIMESTAMPTZ)
        - NEVER use nested aggregate functions like AVG(SUM(...))
        - Use subqueries for complex aggregations
        - Use proper time filtering with PostgreSQL date functions
        - Include appropriate GROUP BY clauses
        - Use LIMIT when showing recent data
        - NEVER use aggregate functions in GROUP BY clause
        - Only reference columns that exist in the available columns list

        Examples of VALID patterns:
        - SELECT AVG(column_name) FROM metric WHERE time >= NOW() - INTERVAL '1 hour'
        - SELECT date_trunc('hour', time), SUM(column_name) FROM metric GROUP BY date_trunc('hour', time)
        - SELECT AVG(daily_total) FROM (SELECT DATE(time), SUM(column_name) as daily_total FROM metric GROUP BY DATE(time)) subq
        - SELECT time, column_name FROM metric ORDER BY time DESC LIMIT 10

        Examples of INVALID patterns (DO NOT USE):
        - SELECT AVG(SUM(column_name)) FROM metric -- INVALID: nested aggregates
        - SELECT SUM(AVG(column_name)) FROM metric -- INVALID: nested aggregates
        - SELECT time, SUM(column_name) FROM metric GROUP BY COUNT(other_column) -- INVALID: aggregate in GROUP BY
        - SELECT nonexistent_column FROM metric -- INVALID: column doesn't exist

        Generate only the SQL query:
        """

        attempt = 0
        while attempt < max_retries:
            logger.info(f"Etapa 2: Tentativa {attempt + 1} de {max_retries} para gerar query SQL...")
            sql_query = self.call_ollama("llama3.2:latest", base_prompt, system_prompt)

            sql_query = re.sub(r'```sql\n?', '', sql_query)
            sql_query = re.sub(r'```\n?', '', sql_query)
            sql_query = sql_query.strip()

            validation_errors = None
            # self.validate_postgres_sql(sql_query, available_columns)
            if not validation_errors:
                logger.info(f"Query SQL gerada com sucesso: {sql_query}")
                return sql_query
            else:
                logger.warning(f"Tentativa {attempt + 1} falhou com erros: {validation_errors}")
                attempt += 1

        logger.error("Falha ao gerar uma query SQL válida após várias tentativas.")
        raise ValueError("Não foi possível gerar uma query SQL válida.")


    def _extract_column_names(self) -> list:
        """Extract column names from the table schema"""
        try:

            columns = []
            if hasattr(self, 'table_schema') and self.table_schema:
                if isinstance(self.table_schema, str):
                    import re
                    column_matches = re.findall(r'(\w+)\s+(?:INTEGER|VARCHAR|TEXT|TIMESTAMPTZ|NUMERIC|FLOAT|DOUBLE)', 
                                            self.table_schema, re.IGNORECASE)
                    columns = column_matches
                
            if not columns:
                try:
                    query = """
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name = 'metric' 
                    ORDER BY ordinal_position
                    """
                    result = self.db_client.execute_query(query)
                    columns = [row[0] for row in result]
                except Exception as e:
                    logger.warning(f"Could not fetch column names: {e}")
                    columns = ['time']
            
            return columns
        except Exception as e:
            logger.error(f"Error extracting column names: {e}")
            return ['time']

    

    def validate_postgres_sql(self,sql_query: str, available_columns: list) -> list:
        errors = []

        try:
            # Define parsing para PostgreSQL
            parsed = sqlglot.parse_one(sql_query, read="postgres")
        except sqlglot.errors.ParseError as e:
            return [f"SQL syntax error (PostgreSQL): {e}"]

        used_columns = set()
        agg_stack = []

        def visit(node, inside_group_by=False):
            # Coleta colunas
            if isinstance(node, exp.Column):
                used_columns.add(node.name)

            # Detecta agregações aninhadas
            if isinstance(node, exp.AggFunc):
                if agg_stack:
                    errors.append("Nested aggregate functions detected.")
                agg_stack.append(node)

            # Detecta agregações dentro de GROUP BY
            if inside_group_by and isinstance(node, exp.AggFunc):
                errors.append("Aggregate function inside GROUP BY clause.")

            # Recursão nos filhos da árvore
            for child in node.args.values():
                if isinstance(child, list):
                    for item in child:
                        visit(item, inside_group_by=inside_group_by)
                elif isinstance(child, exp.Expression):
                    visit(child, inside_group_by=inside_group_by)

            if isinstance(node, exp.AggFunc):
                agg_stack.pop()

        # Visita geral
        visit(parsed)

        # Verificar GROUP BY separadamente
        group_by_exprs = parsed.args.get("group", [])
        for expr in group_by_exprs:
            visit(expr, inside_group_by=True)

        # Verifica colunas não existentes
        missing_columns = [col for col in used_columns if col not in available_columns]
        if missing_columns:
            errors.append(f"Potential missing columns: {missing_columns}")

        return errors

    def interpret_results(self, original_question: str, sql_query: str, 
                         results: list, execution_status: str) -> str:
        system_prompt = """
        Você é um assistente especializado em análise de dados de redes 5G.
        Sua tarefa é interpretar os resultados de consultas SQL e fornecer uma resposta clara em português.
        
        Diretrizes:
        1. Responda em português brasileiro
        2. Seja claro e objetivo
        3. Forneça insights relevantes sobre os dados
        4. Explique o que os números significam no contexto de redes 5G
        5. Se houver problemas com a query, explique de forma compreensível
        6. Use formatação clara para apresentar os dados
        """
        
        if results:
            results_summary = f"Resultados encontrados: {len(results)} registros\n"
            results_summary += f"Primeiros resultados:\n{json.dumps(results[:5], indent=2, default=str)}"
        else:
            results_summary = "Nenhum resultado encontrado ou erro na execução."
        
        prompt = f"""
        Pergunta original do usuário: "{original_question}"
        
        Query SQL executada: {sql_query}
        
        Status da execução: {execution_status}
        
        {results_summary}
        
        Forneça uma resposta completa em português interpretando estes resultados para o usuário.
        Se houver dados, analise-os e forneça insights relevantes.
        Se houver erros, explique o que aconteceu de forma compreensível.
        """
        
        logger.info("Etapa 4: Interpretando resultados...")
        interpretation = self.call_ollama("llama3.2:latest", prompt, system_prompt)
        
        return interpretation

    def process_question(self, question: str):
        start_time = datetime.now()
        try:
            processed_question = self.translate_and_process_question(question)

            sql_query = self.generate_sql_query(processed_question)

            results, execution_status = self.execute_sql_query(sql_query)

            interpretation = self.interpret_results(
                question, sql_query, results, execution_status
            )

            end_time = datetime.now()
            processing_time = (end_time - start_time).total_seconds()
            
            return {
                "success": True,
                "original_question": question,
                "processed_question": processed_question,
                "sql_query": sql_query,
                "execution_status": execution_status,
                "results_count": len(results),
                "results": results,
                "interpretation": interpretation,
                "processing_time_seconds": processing_time,
                "timestamp": start_time.isoformat()
            }

        except Exception as e:
            print(e)
            error_msg = f"Erro no pipeline: {str(e)}"
            logger.error(error_msg)
            
            return {
                "success": False,
                "error": error_msg,
                "original_question": question,
                "timestamp": start_time.isoformat()
            }
        
    

In [8]:
pipe = Pipeline()

perguntas_exemplo = [
        "Quantos assinantes estão registrados atualmente?",
        "Qual é a taxa de sucesso das sessões PDU na última semana?",
        "Mostre o throughput médio de upload e download nas últimas 24 horas",
        "Qual é a taxa de erro de bloco para uplink e downlink hoje?",
        "Quantas sessões ativas existem no momento?"
    ]
for pergunta in perguntas_exemplo:
    resultado = pipe.process_question(pergunta)
    if resultado["success"]:
        print(f"Processamento concluído em {resultado['processing_time_seconds']:.2f}s")
        print(f"Resultados encontrados: {resultado['results_count']}")
        print(f"Query SQL: {resultado['sql_query']}")
        print(f"Interpretação: {resultado['interpretation']}")
    else:
        print(f"Bola furada...")


INFO:__main__:Etapa 1: Traduzindo e processando pergunta...


INFO:__main__:Pergunta processada: "Number of active subscribers in the 5G network."
INFO:__main__:Etapa 2: Tentativa 1 de 3 para gerar query SQL...
INFO:__main__:Query SQL gerada com sucesso: SELECT fivegs_amffunction_rm_registeredsubnbr FROM metric WHERE time >= NOW() - INTERVAL '1 hour'
INFO:__main__:Etapa 3: Executando query SQL...
INFO:__main__:Etapa 4: Interpretando resultados...
INFO:__main__:Etapa 1: Traduzindo e processando pergunta...


Processamento concluído em 269.79s
Resultados encontrados: 63
Query SQL: SELECT fivegs_amffunction_rm_registeredsubnbr FROM metric WHERE time >= NOW() - INTERVAL '1 hour'
Interpretação: **Resposta do Assistente**

Olá! Estou aqui para ajudar a interpretar os resultados da sua consulta SQL.

**Resumo dos Resultados**

A consulta SQL executada foi bem-sucedida e retornou 63 registros. No entanto, é importante notar que os primeiros resultados apresentam apenas cinco registros com o valor "0" para a coluna "fivegs_amffunction_rm_registeredsubnbr". Isso sugere que os registros iniciais podem não estar corretos ou podem ser um erro de carregamento.

**Análise dos Dados**

Para entender melhor o número total de assinantes registrados atualmente, precisamos analisar os dados mais a fundo. A coluna "fivegs_amffunction_rm_registeredsubnbr" representa o número de assinantes registrados no sistema. O valor "0" nos primeiros registros pode ser um erro de carregamento ou uma falha na consulta.

**I

INFO:__main__:Pergunta processada: "Which is the session PDU success rate for the last week in 5G network?"
INFO:__main__:Etapa 2: Tentativa 1 de 3 para gerar query SQL...
INFO:__main__:Query SQL gerada com sucesso: SELECT 
    time,
    AVG(fivegs_smffunction_sm_pdusessioncreationsucc) AS fivegs_smffunction_sm_pdusessioncreationsucc_avg
FROM 
    metric
WHERE 
    time >= NOW() - INTERVAL '1 week'
GROUP BY 
    time
ORDER BY 
    time DESC
LIMIT 10;
INFO:__main__:Etapa 3: Executando query SQL...
INFO:__main__:Etapa 4: Interpretando resultados...
INFO:__main__:Etapa 1: Traduzindo e processando pergunta...


Processamento concluído em 125.46s
Resultados encontrados: 10
Query SQL: SELECT 
    time,
    AVG(fivegs_smffunction_sm_pdusessioncreationsucc) AS fivegs_smffunction_sm_pdusessioncreationsucc_avg
FROM 
    metric
WHERE 
    time >= NOW() - INTERVAL '1 week'
GROUP BY 
    time
ORDER BY 
    time DESC
LIMIT 10;
Interpretação: **Resposta ao Pergunta Original**

A taxa de sucesso das sessões PDU na última semana é um indicador importante para avaliar a eficiência da rede 5G. Com base nos resultados da consulta SQL, podemos analisar os dados e fornecer insights relevantes.

**Análise dos Resultados**

Os resultados mostram que a média de sucesso das sessões PDU na última semana é de aproximadamente **0E-20**, o que significa que cerca de 99,999% das sessões PDU foram bem-sucedidas. Isso é um indicativo muito alto e sugere que a rede 5G está funcionando de forma eficiente.

**Insights Relevantes**

* A alta taxa de sucesso das sessões PDU indica que a rede 5G está lidando bem com o tráfego 

...

Processamento concluído em 122.97s
Resultados encontrados: 0
Query SQL: SELECT 
    AVG(CASE WHEN column_name = 'fivegs_ep_n3_gtp_outdatapktn3upf' THEN value ELSE NULL END) AS avg_upload_throughput,
    AVG(CASE WHEN column_name = 'fivegs_ep_n3_gtp_indatapktn3upf' THEN value ELSE NULL END) AS avg_download_throughput
FROM 
    metric
WHERE 
    time >= NOW() - INTERVAL '24 hour'
ORDER BY 
    time DESC
LIMIT 1;
Interpretação: **Análise dos Resultados**

Infelizmente, não foi possível executar a consulta SQL original devido ao erro "column 'column_name' does not exist". Isso significa que o nome da coluna especificada no filtro `CASE WHEN` não existe na tabela `metric`.

**O que aconteceu?**

A probabilidade é de que o nome da coluna seja incorreto ou não esteja sendo usado corretamente. É possível que a coluna tenha um nome diferente do que foi especificado, ou talvez haja um erro de digitação.

**Sugestões para resolver o problema**

1. Verifique o nome da coluna: Revise a tabela `metr

INFO:__main__:Pergunta processada: "Which is the block error rate for Uplink and Downlink as of today?"
INFO:__main__:Etapa 2: Tentativa 1 de 3 para gerar query SQL...
INFO:__main__:Query SQL gerada com sucesso: SELECT AVG(softmodern_bler_ul), AVG(softmodern_bler_dl) FROM metric WHERE time >= NOW()
INFO:__main__:Etapa 3: Executando query SQL...
INFO:__main__:Etapa 4: Interpretando resultados...
INFO:__main__:Etapa 1: Traduzindo e processando pergunta...


Processamento concluído em 96.72s
Resultados encontrados: 1
Query SQL: SELECT AVG(softmodern_bler_ul), AVG(softmodern_bler_dl) FROM metric WHERE time >= NOW()
Interpretação: **Resposta do Assistente**

Olá! Estou aqui para ajudar a interpretar os resultados da sua consulta SQL.

Infelizmente, não há dados disponíveis para analisar. O resultado da consulta é um array vazio com apenas 1 registro, mas o valor médio de erro de bloco (softmodern_bler_ul e softmodern_bler_dl) está igual a null.

**O que significa "null"?**

Em SQL, o valor null indica que não há dados disponíveis para calcular. Nesse caso, isso significa que não há registros de erros de bloco disponíveis nos últimos minutos.

**Por que isso pode acontecer?**

Existem várias razões pelas quais os dados podem estar faltando:

*   A rede 5G ainda está em desenvolvimento e pode haver problemas técnicos.
*   Os dados podem não estar sendo coletados corretamente ou de forma contínua.
*   Há um problema com a consulta SQL ou com o 

INFO:__main__:Pergunta processada: "Active sessions count in real-time."
INFO:__main__:Etapa 2: Tentativa 1 de 3 para gerar query SQL...
INFO:__main__:Query SQL gerada com sucesso: SELECT AVG(fivegs_smffunction_sm_sessionnbr) FROM metric WHERE time >= NOW()
INFO:__main__:Etapa 3: Executando query SQL...
INFO:__main__:Etapa 4: Interpretando resultados...


Processamento concluído em 94.23s
Resultados encontrados: 1
Query SQL: SELECT AVG(fivegs_smffunction_sm_sessionnbr) FROM metric WHERE time >= NOW()
Interpretação: **Resposta do Assistente**

Olá! Estou aqui para ajudar a interpretar os resultados da sua consulta SQL.

Infelizmente, não há dados disponíveis para fornecer uma resposta clara. O resultado da consulta é um único registro com o valor `null` no campo `avg`, que representa a média das sessões ativas.

**O que significa "null"?**

Em termos de redes 5G, a média das sessões ativas (ou seja, o número de conexões ativas) é um indicador importante para avaliar a capacidade da rede. No entanto, nesse caso, o valor `null` indica que não há dados disponíveis para calcular essa média.

**Por que isso aconteceu?**

É possível que a consulta SQL não tenha encontrado registros relevantes nos últimos minutos (o que é indicado pela condição `time >= NOW()`). Isso pode ocorrer se a rede 5G ainda estiver em processo de configuração ou se não 



        Get the current 5G network performance metrics, including throughput, latency and packet loss rates.

""" 
### Instructions:
Your task is to convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the question create a query to analyze network status.
This query will run on a database whose schema is represented in this string:
        CREATE TABLE IF NOT EXISTS metric (
            time TIMESTAMPTZ NOT NULL PRIMARY KEY, -- Metric timestamp
            fivegs_amffunction_rm_registeredsubnbr INT, -- Number of registered subscribers
            fivegs_amffunction_rm_regmobreq INT, -- Mobility registration update requests
            fivegs_amffunction_rm_regmobsucc INT, -- Successful mobility registration updates
            fivegs_amffunction_rm_regmobfail INT, -- Failed mobility registration updates
            fivegs_smffunction_sm_sessionnbr INT, -- Number of active sessions
            fivegs_smffunction_sm_pdusessioncreationreq INT, -- PDU session creation requests
            fivegs_smffunction_sm_pdusessioncreationsucc INT, -- Successful PDU session creations
            fivegs_ep_n3_gtp_indatapktn3upf INT, -- Uplink throughput on N3 interface
            fivegs_ep_n3_gtp_outdatapktn3upf INT, -- Downlink throughput on N3 interface
            fivegs_upffunction_upf_sessionnbr INT, -- Average number of PDU sessions
            fivegs_smffunction_upf_qos_flow_nbr INT, -- Number of active QoS flows
            fivegs_upffunction_sm_n4sessionreport INT, -- N4 session reports
            fivegs_upffunction_sm_n4sessionreportsucc INT, -- Successful N4 session reports
            fivegs_upffunction_sm_n4sessionestabreq INT, -- N4 session establishment requests
            fivegs_upffunction_sm_n4sessionestabfail INT, -- Failed N4 session establishments
            softmodern_bler_dl INT, -- Block error rate - downlink
            softmodern_bler_ul INT, -- Block error rate - uplink
            softmodern_rsrp INT, -- Received signal power
            pfcp_sessions_active INT -- Used for testing value changes
        );

Based on your instructions and the fact , here is the SQL query I have generated to answer the question create a the number of active and failed sessions:
"""
